# Echo – listens and reflects feelings back, symbolizing understanding.

**Chatbot Skeleton**

In [1]:
# Importing Libraries
import praw
import csv
from datetime import datetime
import os
from tqdm import tqdm
import time
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

# Pre-processing libraries
import os
import pandas as pd
import re
import emoji
import unicodedata
import string
import contractions
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from translatepy import Translator
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

In [2]:
def preprocess_text(raw_text: str) -> str:
    EMOTION_MAP = {
        # 😢 Sad / Negative / Depressed
        "😭": "crying",
        "💔": "broken_heart",
        "😔": "sad",
        "😢": "tears",
        "😞": "disappointed",
        "😓": "anxious",
        "😟": "worried",
        "🙁": "unhappy",
        "😩": "exhausted",
        "😫": "tired",
        "😖": "frustrated",
        "😣": "distressed",
        "🥺": "pleading",
        "😿": "crying_cat",
        "☹️": "frowning",
        
        # 🙂 Neutral / Thinking
        "😐": "neutral",
        "🤔": "thinking",
        "🙄": "eye_roll",
        
        # 😀 Happy / Positive (less emphasized)
        "😂": "laughing",
        "😁": "grinning",
        "😊": "happy",
        "😍": "love",
        "❤": "love",
        "🤗": "hugging",
        "🎉": "celebration"
    }
    
    stop_words = set(stopwords.words('english'))
    translator = Translator()
    
    # ======================
    # Helper Functions
    # ======================
    def expand_contractions(text):
        return contractions.fix(text)
    
    def handle_special_characters(text):
        text = str(text)
        text = re.sub(r'[\u200b-\u200f\u202a-\u202e]', '', text)  # invisible chars
        text = re.sub(r'[❤♡♥]', 'heart', text)
        text = re.sub(r'[★☆✮✯]', 'star', text)
        text = unicodedata.normalize('NFKD', text)
        return text.encode('ascii', 'ignore').decode('ascii')
    
    def replace_emojis(text):
        for emoji_char, label in EMOTION_MAP.items():
            text = text.replace(emoji_char, f" {label}")
        return text
    
    def translate_to_english(text):
        try:
            if not text.strip():
                return text
            result = translator.translate(text, "English")
            translated_text = result.result
            if translated_text.strip().lower() == text.strip().lower():
                return text
            else:
                return translated_text
        except Exception:
            return text
    text = str(raw_text).strip()
    
    # Replace emojis first
    text = replace_emojis(text)
    
    # Translate, expand, handle special chars
    text = text.lower()
    text = translate_to_english(text)
    text = expand_contractions(text)
    text = handle_special_characters(text)
    
    # Remove unwanted punctuation (keep underscores)
    text = re.sub(r"[^\w\s]", " ", text)

    # trimming
    text = ' '.join(text.split()[:400])

    # Tokenize and remove stopwords
    tokens = [t for t in text.split() if t not in stop_words]
    
    return " ".join(tokens)

In [3]:
result = preprocess_text("hello😂 dark place last 2 years reached lowest point reflecting exactly point last 27 years life remaining potentially 40 look like quite happy person younger care world felt grounded reality enjoyed time friends family even though upbringing low socioeconomic even though raised single parent felt motivated felt great things would happen life failed high school squandered serious relationship ever wondered aimlessly 23 playing video games jobs fast forward money 5th year meant 4 year degree repeated 3rd year twice like 50k debt fail degree know man feel like even pass degree want job degree preparing know want know people normally illusion said someone 27 money barely work experience would eyebrows raising want functional human cannot find purpose cannot find anything drive reaching hopes someone maybe similar situation could would light go find purpose")
result

'hello laughing dark place last 2 years reached lowest point reflecting exactly point last 27 years life remaining potentially 40 look like quite happy person younger care world felt grounded reality enjoyed time friends family even though upbringing low socioeconomic even though raised single parent felt motivated felt great things would happen life failed high school squandered serious relationship ever wondered aimlessly 23 playing video games jobs fast forward money 5th year meant 4 year degree repeated 3rd year twice like 50k debt fail degree know man feel like even pass degree want job degree preparing know want know people normally illusion said someone 27 money barely work experience would eyebrows raising want functional human cannot find purpose cannot find anything drive reaching hopes someone maybe similar situation could would light go find purpose'

In [4]:
def compute_severity(text: str) -> float:
    from transformers import AutoTokenizer, AutoModelForSequenceClassification
    import numpy as np

    # Load model globally so we don’t reload every call
    model_name = "cardiffnlp/twitter-roberta-base-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    labels = ['Negative', 'Neutral', 'Positive']
    
    # Valence and arousal mapping
    valence_map = {'Positive': 1.0, 'Neutral': 0.0, 'Negative': -1.0}
    arousal_map = {'Positive': 0.7, 'Neutral': 0.5, 'Negative': 0.8}
    
    def get_sentiment(text):
        encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
        output = model(**encoded_input)
        scores = output.logits.detach().numpy()[0]
        probs = np.exp(scores) / np.exp(scores).sum()
        return dict(zip(labels, probs))
    
    sentiment = get_sentiment(text)

    # Valence & arousal
    valence = (
        sentiment['Positive'] * valence_map['Positive'] +
        sentiment['Neutral']  * valence_map['Neutral'] +
        sentiment['Negative'] * valence_map['Negative']
    )
    arousal = (
        sentiment['Positive'] * arousal_map['Positive'] +
        sentiment['Neutral']  * arousal_map['Neutral'] +
        sentiment['Negative'] * arousal_map['Negative']
    )

    # Raw severity
    raw_severity = -valence * arousal
    
    # Normalize severity between 0 and 1 for easier interpretation
    scaler = MinMaxScaler()
    severity = scaler.fit_transform(np.array(raw_severity).reshape(-1, 1))

    return float(severity[0][0])

In [5]:
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# ==========================
# Load Model Once
# ==========================
model_name = "cardiffnlp/twitter-roberta-base-sentiment"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
labels = ['Negative', 'Neutral', 'Positive']

# ==========================
# Valence & Arousal Mapping
# ==========================
valence_map = {'Positive': 1.0, 'Neutral': 0.0, 'Negative': -1.0}
arousal_map = {'Positive': 0.7, 'Neutral': 0.5, 'Negative': 0.8}

# ==========================
# Single Text Severity
# ==========================
def compute_severity(text: str) -> float:
    encoded_input = tokenizer(text, return_tensors='pt', truncation=True, max_length=512)
    with torch.no_grad():
        output = model(**encoded_input)

    scores = output.logits.detach().numpy()[0]
    probs = np.exp(scores) / np.exp(scores).sum()
    sentiment = dict(zip(labels, probs))

    # Weighted valence & arousal
    valence = (
        sentiment['Positive'] * valence_map['Positive'] +
        sentiment['Neutral']  * valence_map['Neutral'] +
        sentiment['Negative'] * valence_map['Negative']
    )
    arousal = (
        sentiment['Positive'] * arousal_map['Positive'] +
        sentiment['Neutral']  * arousal_map['Neutral'] +
        sentiment['Negative'] * arousal_map['Negative']
    )

    # Raw severity
    raw_severity = -valence * arousal

    # Normalize raw severity to 0–1 using tanh (smooth scaling)
    severity = (np.tanh(raw_severity) + 1) / 2

    return float(severity)


In [6]:
# text = "life seriously worst like cannot say much people start judging say life like but seriously fucked every fucking negative trait life literally wreck trying fix years cannot energy continue everyone seems something going shit wtfwtf lowk want end icl cryingcryingbroken_heartbroken_heartbroken_heart"
text = "I am not feeling well"
severity = compute_severity(text)

print(severity)

0.8175044655799866


In [7]:
# from sklearn.preprocessing import MinMaxScaler
# import numpy as np

# class ConversationSeverityTracker:
#     def __init__(self):
#         self.raw_severities = []

#     def add_message(self, text, compute_raw_severity):
#         # Compute raw severity for this message
#         raw = compute_raw_severity(text)  
#         self.raw_severities.append(raw)

#         # Rescale all severities so far
#         scaler = MinMaxScaler()
#         scaled = scaler.fit_transform(
#             np.array(self.raw_severities).reshape(-1, 1)
#         )

#         # Return the scaled severity for the latest message
#         return float(scaled[-1][0])

# # Usage:
# tracker = ConversationSeverityTracker()

# msg1 = tracker.add_message("I’m gonna die lol", compute_severity)
# msg2 = tracker.add_message("nah just kidding", compute_severity)
# msg3 = tracker.add_message("but honestly I’m exhausted and can’t go on", compute_severity)

# print(msg1, msg2, msg3)  # progressively more accurate severities


In [8]:
import warnings
warnings.filterwarnings('ignore')

from transformers import pipeline

# Emotion detection
emotion_model = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base")

# Topic detection
topic_model = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

emotion = emotion_model(text)[0]['label']
topic = topic_model(text, candidate_labels=["work", "relationship", "family", "health", "finance"])['labels'][0]

Device set to use cpu
Device set to use cpu


In [12]:
import threading
import time
import pandas as pd
import numpy as np
from datetime import datetime
import os
import warnings
warnings.filterwarnings('ignore')
from collections import Counter
import nltk
# nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import json
import random

# Load topic-based responses
with open("responses.json", "r") as f:
    topic_responses = json.load(f)

# -----------------------------
# Load or define severity thresholds
# -----------------------------
p75 = 0.86  # Moderate
p90 = 0.94 # High
p95 = 0.97 # Critical

def assign_risk(sev):
    if sev <= p75:
        return "Low"
    elif sev <= p90:
        return "Moderate"
    elif sev <= p95:
        return "High"
    else:
        return "Critical"

# -----------------------------
# Placeholder functions
# -----------------------------

def alert_helpline(user_id, text, severity):
    print(f"ALERT: Critical risk detected for {user_id} | Severity: {severity:.2f}")
    # Here you can integrate with SMS/email/API to contact help line

# -----------------------------
# Logging function
# -----------------------------
def log_message(user_id, text, processed, severity, risk_level, file_path="output/chat_log_with_severity.csv"):
    df = pd.DataFrame([{
        "timestamp": datetime.utcnow(),
        "user_id": user_id,
        "text": text,
        "Pre_processed_text":processed,
        "severity": severity,
        "risk_level": risk_level
    }])
    
    # If file exists, append without header. If not, write header.
    if not os.path.isfile(file_path):
        df.to_csv(file_path, mode='w', header=True, index=False)
    else:
        df.to_csv(file_path, mode='a', header=False, index=False)

# -----------------------------
# Background severity checker
# -----------------------------
def monitor_severity(user_id, text):
    processed = preprocess_text(text)
    severity = compute_severity(processed)
    risk_level = assign_risk(severity)
    
    # Log the message
    log_message(user_id, text, processed, severity, risk_level)
    
    # Trigger alert if critical
    if risk_level == "Critical":
        alert_helpline(user_id, processed, severity)
    
    return severity, risk_level

# -----------------------------
# Topic detection
# -----------------------------
def detect_topic(text):
    text = text.lower()
    topic_counter = Counter()
    
    # Split text into sentences
    sentences = sent_tokenize(text)
    
    for sent in sentences:
        words = word_tokenize(sent)
        
        # Count mentions for each topic
        if any(word in words for word in ["work", "job", "boss", "office"]):
            topic_counter["work"] += 1
        if any(word in words for word in ["family", "parents", "siblings"]):
            topic_counter["family"] += 1
        if any(word in words for word in ["school", "study", "exam", "university", "teacher", "assignment", "homework"]):
            topic_counter["study"] += 1
    
    # Print counts of all detected topics
    # print("Topic counts:", dict(topic_counter))
    
    if topic_counter:
        # Return the topic with highest count
        main_topic = topic_counter.most_common(1)[0][0]
    else:
        main_topic = "general"
    
    return main_topic

# -----------------------------
# Chatbot main loop (simplified)
# -----------------------------
def chatbot():
    user_id = "user123"
    print("Chatbot: Hello! I'm here to chat with you. Type 'exit' to end.")
    
    while True:
        user_input = input("User: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye! Take care.")
            break
        
        # Start background thread for severity analysis
        severity_thread = threading.Thread(target=monitor_severity, args=(user_id, user_input))
        severity_thread.start()
        
        # Detect topic
        topic = detect_topic(user_input)
        # print(f"[DEBUG] Detected topic: {topic}")
        
        # Choose a random response based on topic
        if topic in topic_responses:
            response = random.choice(topic_responses[topic])
        else:
            response = random.choice(topic_responses["general"])
        
        print(f"Chatbot: {response}")


# -----------------------------
# Start chatbot
# -----------------------------
if __name__ == "__main__":
    chatbot()
     

Chatbot: Hello! I'm here to chat with you. Type 'exit' to end.


User:  how are you?


[DEBUG] Detected topic: general
Chatbot: It's okay to feel that way sometimes. Would you like to talk about it?


User:  I am having fight with my wife


[DEBUG] Detected topic: general
Chatbot: It's okay to feel that way sometimes. Would you like to talk about it?


User:  I am parents don't like me


[DEBUG] Detected topic: family
Chatbot: Sometimes family conflicts take an emotional tollâ€”how are you coping?


User:  exit


Chatbot: Goodbye! Take care.
